In [1]:
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
from pandas import Series, DataFrame
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime as dt
import requests


In [2]:
eng=create_engine('sqlite:///mta_data.db')

In [3]:
#Reqires an 'mta_data.db' file. I can't post the file itself to GitHub, but you can use get_mta with parameter "(21|20|19|18)"
query="select *, case when time like '00%' or time like '01%' or time like '02%' or time like '03% 'then 'kept' else 'not midnight' end as time_keep from mta_data where time_keep is not 'not midnight'"

mta=pd.read_sql(query,eng)

mta.to_csv('mtacp1.csv')

In [4]:
#SAVEPOINT #1

mta=pd.read_csv('mtacp1.csv')
mta.drop(['Unnamed: 0'], axis=1, inplace=True)

mta['DATE']=pd.to_datetime(mta['DATE'])

mta.drop_duplicates(inplace=True)

In [5]:
monthlist=[]
i=0
for item in mta['DATE']:
    m=str(item).split(" ")[0].split('-')[0:2]
    monthlist.append(m[0]+'/'+m[1])
mta['month']=monthlist

In [6]:
#mta=mta.groupby(['SCP',"STATION"]).sort_values('DATE', ascending=False)

mta[["PREV_DATE", "PREV_EXITS"]] = (mta.groupby(["C/A", "UNIT", "SCP", "STATION"])[["DATE", "EXITS"]].shift(1))
mta["PREV_ENTRIES"] = (mta.groupby(["C/A", "UNIT", "SCP", "STATION"])[["ENTRIES"]].shift(1))
mta.drop(['C/A','UNIT','LINENAME','TIME','DESC','time_keep','DIVISION'], inplace=True, axis=1)


In [ ]:
def get_daily_entries(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    
    if counter < 0:
        counter = -counter  # adjust for "reverse" counter
        
    if counter > max_counter:
        # Maybe counter was reset, so it may make sense to take the minimum
        print(f'entries: {row["ENTRIES"]} <-- {row["PREV_ENTRIES"]}')
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
        
    if counter > max_counter:
        # If we still get a counter that is too big, set to zero
        return 0
    
    return counter

def get_daily_exits(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    
    if counter < 0:
        counter = -counter  # adjust for "reverse" counter
        
    if counter > max_counter:
        # Maybe counter was reset, so it may make sense to take the minimum
        print(f'exits: {row["EXITS"]} <-- {row["PREV_EXITS"]}')
        counter = min(row["EXITS"], row["PREV_EXITS"])
        
    if counter > max_counter:
        # If we still get a counter that is too big, set to zero
        return 0
    
    return counter

mta["DAILY_ENTRIES"] = mta.apply(get_daily_entries, axis=1, max_counter=500000)
mta["DAILY_EXITS"] = mta.apply(get_daily_exits, axis=1, max_counter=500000)

entries: 197281 <-- 1891026.0
entries: 327683 <-- 50346231.0
entries: 1605012 <-- 6536.0
entries: 200 <-- 1605012.0
entries: 558268 <-- 594.0
entries: 13 <-- 558268.0
entries: 909042686 <-- 1814.0
entries: 1883435 <-- 198211.0
entries: 50346223 <-- 327690.0
entries: 1601701 <-- 3023.0
entries: 1301 <-- 12519606.0
entries: 12517625 <-- 94.0
entries: 557981 <-- 289.0
entries: 3275794 <-- 198873.0
entries: 196879 <-- 3277263.0
entries: 303653721 <-- 1008.0
entries: 87 <-- 303654987.0
entries: 11850991 <-- 5078.0
entries: 432 <-- 11851565.0
entries: 141 <-- 12518130.0
entries: 3273393 <-- 196879.0
entries: 303651837 <-- 87.0
entries: 12917049 <-- 1444108.0
entries: 11848060 <-- 1920.0
entries: 19469567 <-- 687983549.0
entries: 687970645 <-- 19471254.0
entries: 7486076 <-- 13545.0
entries: 1217 <-- 7489966.0
entries: 692665 <-- 462.0
entries: 39 <-- 692665.0
entries: 104109548 <-- 2950.0
entries: 393412 <-- 104110962.0
entries: 1925320 <-- 107.0
entries: 19461582 <-- 687974832.0
entries: 74

entries: 2026074 <-- 539.0
entries: 13197311 <-- 49040.0
entries: 2843389 <-- 2845.0
entries: 347 <-- 2844694.0
entries: 6235577 <-- 15436306.0
entries: 15439183 <-- 6237156.0
entries: 4001818 <-- 4086.0
entries: 120 <-- 4001999.0
entries: 2841022 <-- 807.0
entries: 2172367 <-- 1834.0
entries: 190 <-- 2172453.0
entries: 500477 <-- 232.0
entries: 38 <-- 500614.0
entries: 1928912 <-- 7326.0
entries: 558 <-- 1930759.0
entries: 587439 <-- 2240.0
entries: 7 <-- 587459.0
entries: 6232884 <-- 15439183.0
entries: 4000555 <-- 1593.0
entries: 141765966 <-- 5021.0
entries: 2171519 <-- 819.0
entries: 4259475 <-- 2296937.0
entries: 2293808 <-- 4259662.0
entries: 500288 <-- 38.0
entries: 1923889 <-- 2368.0
entries: 587366 <-- 7.0
entries: 3039754 <-- 1747.0
entries: 55 <-- 3039862.0
entries: 1029713 <-- 1668.0
entries: 262201 <-- 1029713.0
entries: 4258131 <-- 2295037.0
entries: 1505226 <-- 862.0
entries: 36 <-- 1505300.0
entries: 885806 <-- 312.0
entries: 13 <-- 885806.0
entries: 39697 <-- 2237810.

entries: 869129 <-- 505.0
entries: 26726 <-- 13703909.0
entries: 13704636 <-- 26726.0
entries: 117445461 <-- 720945.0
entries: 524291 <-- 0.0
entries: 2867425 <-- 2727.0
entries: 300 <-- 2867974.0
entries: 1184607 <-- 256.0
entries: 0 <-- 524288.0
entries: 524288 <-- 0.0
entries: 713176 <-- 1341.0
entries: 642823 <-- 1486.0
entries: 811813 <-- 6473.0
entries: 625 <-- 812790.0
entries: 2292771 <-- 1295.0
entries: 245963 <-- 631047402.0
entries: 631046289 <-- 246398.0
entries: 8091761 <-- 2537.0
entries: 6438538 <-- 1136719297.0
entries: 3142970 <-- 1434.0
entries: 117443282 <-- 524303.0
entries: 3334339 <-- 358.0
entries: 2195833 <-- 1617.0
entries: 12000950 <-- 250.0
entries: 5323786 <-- 1666.0
entries: 5122357 <-- 215.0
entries: 2515682 <-- 873332.0
entries: 9678207 <-- 245.0
entries: 6657750 <-- 440.0
entries: 5373111 <-- 586.0
entries: 2835616 <-- 959.0
entries: 3298459 <-- 1030.0
entries: 8631860 <-- 2637.0
entries: 4008094 <-- 594.0
entries: 656063 <-- 2496.0
entries: 2865821 <-- 

entries: 1200257 <-- 1023.0
entries: 98 <-- 1200414.0
entries: 2392777 <-- 48.0
entries: 514028 <-- 2556276.0
entries: 2555909 <-- 514061.0
entries: 6869496 <-- 4238.0
entries: 425 <-- 6871440.0
entries: 235110428 <-- 995.0
entries: 0 <-- 524288.0
entries: 918898 <-- 600.0
entries: 1003301 <-- 651.0
entries: 6665008 <-- 1086.0
entries: 514565 <-- 2456.0
entries: 563270 <-- 109.0
entries: 9 <-- 563308.0
entries: 572170 <-- 542.0
entries: 1199775 <-- 340.0
entries: 513735 <-- 2556034.0
entries: 6866150 <-- 425.0
entries: 6362735 <-- 6.0
entries: 794762 <-- 1263.0
entries: 5112443 <-- 376.0
entries: 4362545 <-- 831.0
entries: 28 <-- 4362546.0
entries: 563204 <-- 33.0
entries: 8235039 <-- 1745.0
entries: 136 <-- 8235039.0
entries: 2090862 <-- 295.0
entries: 18 <-- 2090862.0
entries: 1098262 <-- 349.0
entries: 16 <-- 1098379.0
entries: 9259431 <-- 918703.0
entries: 4362545 <-- 28.0
entries: 39596 <-- 6118274.0
entries: 8234589 <-- 731.0
entries: 997952 <-- 701.0
entries: 8797792 <-- 51.0
en

entries: 8289149 <-- 12327.0
entries: 108 <-- 8294888.0
entries: 2298780 <-- 18352.0
entries: 1457 <-- 2300797.0
entries: 2057098 <-- 411291.0
entries: 395874 <-- 2064683.0
entries: 22737699 <-- 37903.0
entries: 2869 <-- 22737699.0
entries: 624755 <-- 30.0
entries: 7606727 <-- 56.0
entries: 1297159 <-- 131673.0
entries: 655365 <-- 1297159.0
entries: 131141 <-- 655390.0
entries: 167811292 <-- 524297.0
entries: 5357020 <-- 4487.0
entries: 386 <-- 5359422.0
entries: 1450517 <-- 15844.0
entries: 1797 <-- 1455622.0
entries: 8278392 <-- 1926.0
entries: 2343425 <-- 598.0
entries: 2286371 <-- 6652.0
entries: 2038705 <-- 398779.0
entries: 22718970 <-- 12071.0
entries: 1296768 <-- 131328.0
entries: 7063902 <-- 19253.0
entries: 959 <-- 7066197.0
entries: 1823628 <-- 16958.0
entries: 552 <-- 1824508.0
entries: 5353041 <-- 386.0
entries: 584932 <-- 4240.0
entries: 325 <-- 585144.0
entries: 825548 <-- 6748.0
entries: 272 <-- 825716.0
entries: 1506661752 <-- 13297.0
entries: 170 <-- 1506660940.0
entr

entries: 3367061 <-- 1115.0
entries: 29 <-- 3367892.0
entries: 1083367 <-- 1909.0
entries: 26 <-- 1084013.0
entries: 956668 <-- 3027.0
entries: 28 <-- 957481.0
entries: 6192434 <-- 4327.0
entries: 1121333028 <-- 1872.0
entries: 2025849359 <-- 1674.0
entries: 4706395 <-- 4612.0
entries: 50331648 <-- 589824.0
entries: 589824 <-- 50331648.0
entries: 1055053 <-- 9215.0
entries: 152941186 <-- 3035.0
entries: 3272121 <-- 2219.0
entries: 5487749 <-- 1573.0
entries: 5678883 <-- 5168.0
entries: 10 <-- 5679419.0
entries: 117603846 <-- 123793690.0
entries: 123785355 <-- 117604995.0
entries: 17283338 <-- 10796.0
entries: 1325940440 <-- 139438226.0
entries: 3298348 <-- 7975.0
entries: 3359196 <-- 38.0
entries: 1075941 <-- 40.0
entries: 947739 <-- 48.0
entries: 2728637 <-- 5217.0
entries: 107 <-- 2728637.0
entries: 4283137 <-- 5576.0
entries: 114 <-- 4283137.0
entries: 50345125 <-- 196628.0
entries: 50331648 <-- 589824.0
entries: 117440512 <-- 458752.0
entries: 5675546 <-- 1627.0
entries: 3597419 <-

entries: 7542484 <-- 5043.0
entries: 105 <-- 7542484.0
entries: 2338090 <-- 526421.0
entries: 1369523 <-- 11941.0
entries: 999 <-- 1370664.0
entries: 33621828 <-- 2162976.0
entries: 36977714 <-- 3730.0
entries: 1442561158 <-- 78280834.0
entries: 78260066 <-- 1442544807.0
entries: 7270424 <-- 1862340166.0
entries: 1862330382 <-- 7277226.0
entries: 50514958 <-- 33768028.0
entries: 33754095 <-- 50516744.0
entries: 140469233 <-- 1594436339.0
entries: 1594421086 <-- 140471461.0
entries: 1108412174 <-- 17610.0
entries: 930 <-- 1108422349.0
entries: 18327896 <-- 16405.0
entries: 181 <-- 18340503.0
entries: 3548861 <-- 1329.0
entries: 643649 <-- 620.0
entries: 5852537 <-- 527465.0
entries: 524606 <-- 5852785.0
entries: 57020291 <-- 619092.0
entries: 592426 <-- 57022793.0
entries: 3396215 <-- 2066.0
entries: 117848085 <-- 461905.0
entries: 519749 <-- 123.0
entries: 4674268 <-- 1487.0
entries: 3345866 <-- 6433.0
entries: 7535085 <-- 4695.0
entries: 1358361 <-- 1588.0
entries: 1442584351 <-- 7826

entries: 745126 <-- 184611933.0
entries: 184608357 <-- 746094.0
entries: 2896392 <-- 4657.0
entries: 238 <-- 2898346.0
entries: 732770 <-- 143000.0
entries: 132263 <-- 737704.0
entries: 8863914 <-- 7156.0
entries: 644107 <-- 2792.0
entries: 1221 <-- 649233.0
entries: 651217 <-- 2665.0
entries: 68459931 <-- 198519.0
entries: 36941307 <-- 2296.0
entries: 858979 <-- 594.0
entries: 1277 <-- 863113.0
entries: 2354194 <-- 7515.0
entries: 1017 <-- 2357557.0
entries: 4887309 <-- 9953.0
entries: 12 <-- 4887309.0
entries: 9427308 <-- 11140.0
entries: 102381720 <-- 1065584.0
entries: 7582617 <-- 7474.0
entries: 1057270 <-- 10035.0
entries: 6061534 <-- 30941715.0
entries: 75909 <-- 16787286.0
entries: 16779493 <-- 81157.0
entries: 2495527 <-- 637584683.0
entries: 637574909 <-- 2502854.0
entries: 1577619900 <-- 140399438.0
entries: 7475455 <-- 9878.0
entries: 259 <-- 7480462.0
entries: 458752 <-- 683737068.0
entries: 13652884 <-- 17636.0
entries: 7 <-- 13653544.0
entries: 10460989 <-- 2707.0
entrie

In [ ]:
mta.to_csv('mtacp2.csv')

In [ ]:
#SAVEPOINT #2

mta=pd.read_csv('mtacp2.csv')
mta.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
#Used to get a latitude-longitude pair.

#0.01 degrees lat/long ~= 1.111 km
def geocode(address):
    link='https://maps.googleapis.com/maps/api/geocode/json?address='
    key='&key=<Put Google API Key here>'
    address.replace(' ', '+')
    r=requests.get(link+address+key)
    r=r.text
    r=r.split('geometry')[1].split('GEOMETRIC_CENTER')[0]
    lat=float(re.findall('lat.+?([-0-9.]+)',r)[0])
    lng=float(re.findall('lng.+?([-0-9.]+)',r)[0])
    return([lat,lng])

In [ ]:
mta.dropna(inplace=True)

In [ ]:
mta=mta.groupby(['STATION','month']).sum()
mta.drop(['ENTRIES','EXITS','PREV_EXITS','PREV_ENTRIES'], axis=1, inplace=True)
mta.rename(columns={'DAILY_ENTRIES':'entries','DAILY_EXITS':'exits','STATION':'station'}, inplace=True)

In [ ]:
mta.to_csv('mtacp3.csv')

In [ ]:
#SAVEPOINT #3

mta=pd.read_csv('mtacp3.csv')

In [ ]:
stat_coords={}

for station in mta['station']:
    stat_coords[station]=[]

for item in stat_coords:
    coords=geocode(item+" station, NY")
    stat_coords[item]=[coords[0],coords[1]]
stat_coords



In [ ]:
coord_df=pd.DataFrame.from_dict(stat_coords, orient='index')
coord_df.rename(columns={0:'lat',1:'lng'}, inplace=True)
coord_df.to_csv('stat_coords.csv')
coord_df=pd.read_csv('stat_coords.csv')
coord_df.rename(columns={'Unnamed: 0':'station'}, inplace=True)
coord_df

In [ ]:
address_list=[]
for item in stat_coords:
    address_list.append(item)

old_school_lat=[]
old_school_lng=[]

for i in range(len(mta)):
    #print(coord_df.iloc[i]['station'])
    
    j=0
    for item in address_list:
        if mta.iloc[i]['station']==item:
            break
        j+=1
    old_school_lat.append(coord_df.iloc[j]['lat'])
    old_school_lng.append(coord_df.iloc[j]['lng'])
old_school_lat

In [ ]:
mta['lat']=old_school_lat
mta['lng']=old_school_lng
mta

In [ ]:
mta.to_csv('mtacp4.csv')

In [ ]:

#SAVEPOINT #4

mta=pd.read_csv('mtacp4.csv')
mta.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1, inplace=True)

In [ ]:
mta

In [ ]:
totaldf=mta.groupby('month').sum('entries')
totaldf.drop(['lat','lng', 'exits'],axis=1, inplace=True)
totaldf.drop(['2018/12','2019/01','2021/07'],inplace=True)
totaldf.loc['2020/10']['entries']=(totaldf.loc['2020/09']['entries']+totaldf.loc['2020/11']['entries'])/2

In [ ]:
plt.figure(figsize=(20,10)) 
plt.title('MTA ridership over time')
plt.xlabel('Date')
plt.ylabel('Number of Monthly Riders (millions)')
plt.grid()
plt.xticks(rotation=70)
plt.yticks(range(0,500,50))
plt.plot(totaldf.index,totaldf['entries']/1000000)


In [ ]:
count_dict={}
for item in range(len(mta['station'])):
    count_dict[mta.iloc[item]['station']]=0
for item in range(len(mta['station'])):
    count_dict[mta.iloc[item]['station']]+=1
    
count_frame=pd.DataFrame.from_dict(count_dict, orient='index')


count_frame.to_csv('countframe.csv')
count_frame=pd.read_csv('countframe.csv')
count_frame.rename(columns={'0':'count', 'Unnamed: 0':'station'}, inplace=True)

count_list=[]
for item in count_frame['count']:
    count_list.append(item)

In [ ]:
mta2=mta.drop(['month','entries','exits','lat','lng'],axis=1)
mta2.drop_duplicates('station',inplace=True)
mta2['count']=count_list

In [ ]:

#Calculates some values I use in my analysis.

min_val=[]
max_val=[]
growth_rate=[]
pct_of_precovid=[]
min_vs_precovid=[]
for statname in mta2['station']:
    
    tempmin=1e1337

    temp_exits=[]
    for item in mta[mta['station']==statname]['exits']:
        if int(item)<tempmin:
            tempmin=item

    min_val.append(tempmin)
    max_val.append(mta[(mta['month']=='2020/02')&(mta['station']==statname)]['exits'])
    try:
        jun=int(mta[(mta['month']=='2020/06')&(mta['station']==statname)]['exits'])
    except:
        jun=0
    try:
        feb=int(mta[(mta['month']=='2020/02')&(mta['station']==statname)]['exits'])
    except:
        feb=0
    try:
        apr=int(mta[(mta['month']=='2020/04')&(mta['station']==statname)]['exits'])
    except:
        apr=0
    
    try:
        growth_rate.append(((jun/(tempmin+1)-1)*100))
    except:
        growth_rate.append(-1337)
        
    try:
        pct_of_precovid.append(jun/feb*100)
    except:
        pct_of_precovid.append(-1337)
        
    try:
        min_vs_precovid.append(tempmin/feb*100)
    except:
        min_vs_precovid.append(-1337)
        
    
    

mta2['min_val']=min_val
mta2['growth_rate']=growth_rate
mta2['pct_of_precovid']=pct_of_precovid
mta2['min_vs_precovid']=min_vs_precovid





In [ ]:
mta2[mta2['pct_of_precovid']!=-1337]

In [ ]:
#mta.to_csv(mta_organized)

In [ ]:
#Used to create some plots.

def mtaplot(xSet, ySet, sizeDat, title):
    img = plt.imread("osm_bw.png")
    #fig, ax = plt.subplots()
    
    datsize=[]
    for n in sizeDat:
        datsize.append(n/100000)
    
    
    plt.figure(figsize=(20,20)) 
    plt.xlabel('Longtitude')
    plt.ylabel('Latitude')
    plt.title(title)
    plt.ylim(40.5,40.95)
    plt.xlim(-74.15,-73.7)
    #plt.gca().set_aspect('equal', adjustable='box')

    plt.scatter(xSet, ySet, s=datsize, c='b', alpha=0.5)
    plt.draw()
    plt.imshow(img, extent=[-74.15,-73.7, 40.5,40.95])

In [ ]:
mtaplot(mta['lng'],mta['lat'],mta['entries']*2, 'All-time highest traffic per MTA station')

In [ ]:
mta=pd.read_csv('mta_organized.csv')
#Remove stations with incomplete data
mta=mta[mta['count']>31]
mta2=pd.read_csv('mta_names.csv')
mta2.drop(['Unnamed: 0','month','entries','exits','traffic','exit_less_entry','exit_over_entry','latitude','longitude','count'], axis=1, inplace=True)
mta2.drop_duplicates(['station'],inplace=True)
mta2.reset_index(inplace=True)
mta2.drop(['index'], axis=1, inplace=True)

In [ ]:
rankframe={}

yyyy=2018
m=12

while yyyy<2021 or m<7:
    mm=str(m)
    if m<10:
        mm='0'+mm
    
    mth=str(yyyy)+'/'+mm
    
    m+=1
    if m>12:
        m=1
        yyyy+=1
    
    frame=mta[mta['month']==mth]
    frame.sort_values('entries', inplace=True, ascending=False)
    agg=[' ']
    for i in range(0,len(frame['station'])):
        item=frame['station'].iloc[i]
        agg.append(item+', '+str(int(frame['entries'].iloc[i])))
        
    rankframe[mth]=agg
    
rankframe=pd.DataFrame.from_dict(rankframe) 

In [ ]:
top25=rankframe.head(26)
t25=[]
t25v=[]

for item in top25['2021/06']:
    try:
        t25.append(item.split(', ')[0])
        t25v.append(item.split(', ')[1])
    except:
        print(item)
        
t25.pop(0)

In [ ]:
min_val=[]
growth_rate=[]
pct_of_precovid=[]
min_vs_precovid=[]
for statname in mta2['station']:
    
    tempmin=1e1337

    temp_exits=[]
    for item in mta[mta['station']==statname]['exits']:
        if int(item)<tempmin:
            tempmin=item

    min_val.append(tempmin)
    max_val.append(mta[(mta['month']=='2020/02')&(mta['station']==statname)]['exits'])
    jun=int(mta[(mta['month']=='2020/06')&(mta['station']==statname)]['exits'])
    feb=int(mta[(mta['month']=='2020/02')&(mta['station']==statname)]['exits'])
    apr=int(mta[(mta['month']=='2020/04')&(mta['station']==statname)]['exits'])
    growth_rate.append(((jun/(tempmin+1)-1)*100))
    pct_of_precovid.append(jun/feb*100)
    min_vs_precovid.append(tempmin/feb*100)

mta2['min_val']=min_val
mta2['growth_rate']=growth_rate
mta2['pct_of_precovid']=pct_of_precovid
mta2['min_vs_precovid']=min_vs_precovid




In [ ]:
mta3=pd.read_csv('mta2.csv')
mta3.drop(['Unnamed: 0'], axis=1, inplace=True)

mask=[]
for item in mta3['station']:
    print(item)
    if item in t25:
        mask.append(True)
    else:
        mask.append(False)
        
mta3['mask']=mask



In [ ]:
mta3=mta3[mta3['mask']]
mta3['riders']=t25v

In [ ]:
lat=[]
lng=[]
for item in mta3['station']:
    coords=geocode(item+'station ny')
    lat.append(coords[0])
    lng.append(coords[1])
mta3['lat']=lat
mta3['lng']=lng



In [ ]:
q1=mta3.pct_of_precovid.quantile(0.25)
q3=mta3.pct_of_precovid.quantile(0.75)
iqr15=(q3-q1)*1.5

#Drop outliers
mta3=mta3[(mta3['pct_of_precovid']<q3+iqr15)&(mta3['pct_of_precovid']>q1-iqr15)]

mta3=mta3[mta3['lng']<-73.73]

mta3.sort_values('pct_of_precovid', ascending=False, inplace=True)
mta3.reset_index(inplace=True)
mta3.drop(['index'], inplace=True, axis=1)
mta3

mta4=mta3

mta4.sort_values('riders', ascending=False, inplace=True)
mta4.reset_index(inplace=True)
mta4.drop(['index'], inplace=True, axis=1)
mta4


t5=mta3.head(5)




# In the cell below, dot size represents raw ridership, while dot color represents ridership recovery. Red represents low(~10%-25%) recovery, yellow represents average(25-35%) recovery, green represents good(35-60%) recovery.

In [ ]:


mth='2020/06'

frame=mta3


manhattan='12,308,929'

img = plt.imread("osm_bw.png")
#fig, ax = plt.subplots()
sd=frame['riders'].astype(int)*80
# names=[
# 'Atlantic Av. - Barclays Ctr.\n(53.21% recovery,\n1,505,446 riders)',
# 'W 231 St/Broadway\n(41.07% recovery,\n2,133,764 riders)',
# 'Church Av.\n(31.35% recovery,\n1,367,055 riders)',
# '125 St\n(31.19% recovery,\n2,655,680 riders)',
# 'Flushing Main St.\n(31.10% recovery,\n1,353,677 riders)'
# ]



datsize=[]
for n in sd:
    datsize.append(n/100000)


plt.figure(figsize=(20,20)) 
plt.xlabel('Longtitude')
plt.ylabel('Latitude')
plt.title("MTA ridership recovery across New York in June 2021")
plt.ylim(40.5,40.95)
plt.xlim(-74.15,-73.7)
#len(mta3['station'])
for i in range(len(frame['station'])):
    q=frame['pct_of_precovid'].iloc[i]
    red=(255-q*2.55)/255
    gre=q*2.55/255
    if gre>1:
        gre=1
    if red<0:
        red=0
    #s=float(frame['pct_of_precovid'].iloc[i])*85
    plt.scatter(frame['lng'].iloc[i], frame['lat'].iloc[i], s=float(frame['riders'].iloc[i])/1000, color=(red, gre, 0), alpha=0.5)
    plt.plot(frame['lng'].iloc[i], frame['lat'].iloc[i], 'o', c='#0000bb')
    #plt.text(frame['lng'].iloc[i]+0.011, frame['lat'].iloc[i]+0.008, names[i], fontsize=16, color='#dd3333', weight='bold')

#plt.scatter(frame['lng'], frame['lat'], s=datsize, c='b', alpha=0.5)
plt.imshow(img, extent=[-74.15,-73.7, 40.5,40.95], alpha=0.5)

plt.show()

#plt.savefig('mta_best_recovery_06-2021.png')

In [ ]:
mta=